In [0]:
import os
import random
import pathlib
import tempfile
import time
import copy
import torch.optim.lr_scheduler as lr_scheduler
from sklearn.metrics import classification_report
import numpy as np
from functools import partial
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torchvision.models import resnet18, ResNet18_Weights
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset, Subset
import matplotlib.pyplot as plt
import pandas as pd

In [0]:
BASE_DIR= pathlib.Path('/Volumes/pmr_dev/lead_ingots/lead_ingot_images')

image_dir= BASE_DIR / 'Ingot'
label_dir= BASE_DIR / 'Labels/Labels.csv'
file_list = pd.read_csv(label_dir)



In [0]:
display(file_list.head(1))

In [0]:
# Define the function to encode labels
def encode_labels(file_list):
    # Extract image names
    image_names = file_list['Image']
    
    # Combine fault categories into a single label
    # Create a label string by concatenating fault category names where the value is True
    labels = file_list.drop(columns=['Image']).apply(
        lambda row: '_'.join(row.index[row == True]), axis=1
    )
    
    # Encode the labels using LabelEncoder
    le = LabelEncoder()
    encoded_labels = le.fit_transform(labels)
    
    # Create a dictionary mapping image names to encoded labels
    label_dict = dict(zip(image_names, encoded_labels))
    
    return label_dict, le

# Encode the labels
label_dict, label_encoder = encode_labels(file_list)

In [0]:
# Print the mapping of label numbers to fault categories
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Label Number to Fault Category Mapping:")
for fault_category, label_number in label_mapping.items():
    print(f"{label_number}: {fault_category}")

In [0]:
# Calculate and print the frequencies of the different classes
label_counts = pd.Series(label_dict.values()).value_counts()
print("Frequencies of the different classes:")
print(label_counts)

In [0]:
# Function to open and display an image
def open_image(image_name):
    image_path = image_dir / image_name
    image = Image.open(image_path)
    image.show()
    return image.size

# Example usage
example_image_name = file_list.loc[0, 'Image']
image_size = open_image(example_image_name)
print(f"Image size: {image_size}")


In [0]:
# Custom Dataset Class
class LeadIngotDataset(Dataset):
    def __init__(self, image_paths, images, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.images = images  
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

In [0]:
transform = transforms.Compose([
    transforms.Resize((224, 224)), #alexnet, vgg16 input size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) #imagenet noramlization
])

In [0]:
batch_size= 32

dataset= SteelDefectDataset(file_list, label_dict, transform)

train_val_indices, test_indices = train_test_split(
    range(len(dataset)), test_size=0.2, stratify=list(label_dict.values()), random_state=42
)

# Then split train+val into train and validation sets
train_indices, val_indices = train_test_split(
    train_val_indices, test_size=0.125, stratify=[label_dict[dataset.image_files[i].name] for i in train_val_indices], random_state=42
)

# Create Subset datasets
train_dataset = Subset(dataset, train_indices)
val_dataset = Subset(dataset, val_indices)
test_dataset = Subset(dataset, test_indices)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)

# Print statistics
train_labels = [label_dict[dataset.image_files[i].name] for i in train_indices]
val_labels = [label_dict[dataset.image_files[i].name] for i in val_indices]
test_labels = [label_dict[dataset.image_files[i].name] for i in test_indices]

print(f"Train set size: {len(train_dataset)} frequenties: {torch.bincount(torch.tensor(train_labels))}")
print(f"Validation set size: {len(val_dataset)} frequenties: {torch.bincount(torch.tensor(val_labels))}")
print(f"Test set size: {len(test_dataset)} frequenties: {torch.bincount(torch.tensor(test_labels))}")

In [0]:
import torch.nn as nn
class simpleCNN (torch.nn.Module):
    def __init__(self):
        super(simpleCNN, self).__init__()
        self.conv1 = torch.nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = torch.nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv3 = torch.nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv4 = torch.nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv5 = torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.conv6 = torch.nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.conv7 = torch.nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.conv8 = torch.nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.conv9 = torch.nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.conv10 = torch.nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1)

        self.bn1 = torch.nn.BatchNorm2d(16)
        self.bn2 = torch.nn.BatchNorm2d(16)
        self.bn3 = torch.nn.BatchNorm2d(32)
        self.bn4 = torch.nn.BatchNorm2d(32)
        self.bn5 = torch.nn.BatchNorm2d(64)
        self.bn6 = torch.nn.BatchNorm2d(64)
        self.bn7 = torch.nn.BatchNorm2d(128)
        self.bn8 = torch.nn.BatchNorm2d(128)
        self.bn9 = torch.nn.BatchNorm2d(256)
        self.bn10 = torch.nn.BatchNorm2d(256)

        self.fc1 = torch.nn.Linear(256, 10)
        self.fc2 = torch.nn.Linear(10, 6)

        self.max_pool = torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.global_avg_pool = torch.nn.AvgPool2d(kernel_size=14)
        self.relu = torch.nn.ReLU()
        self.flatten = torch.nn.Flatten()

    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.max_pool(x)
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.relu(self.bn4(self.conv4(x)))
        x = self.max_pool(x)
        x = self.relu(self.bn5(self.conv5(x)))
        x = self.relu(self.bn6(self.conv6(x)))
        x = self.max_pool(x)
        x = self.relu(self.bn7(self.conv7(x)))
        x = self.relu(self.bn8(self.conv8(x)))
        x = self.max_pool(x)
        x = self.relu(self.bn9(self.conv9(x)))
        x = self.relu(self.bn10(self.conv10(x)))
        x = self.global_avg_pool(x)
        x = self.flatten(x)  
        x = self.fc1(x)
        x = self.fc2(x)

        
        return x

In [0]:
model=simpleCNN()

#device = torch.device("cuda:0")
device = torch.device("cpu")

model = model.to(device)

In [0]:
#x= torch.ones((64,3,224,224))
#model(x).shape

In [0]:
import time
import torch.nn as nn

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

def train(model, num_epochs, train_dl, valid_dl, patience):
    loss_hist_train = [0] * num_epochs
    accuracy_hist_train = [0] * num_epochs
    loss_hist_valid = [0] * num_epochs
    accuracy_hist_valid = [0] * num_epochs
    patience_counter = 0  # Initialize patience counter
    best_model_wts = model.state_dict()  # Initialize best model weights

    for epoch in range(num_epochs):
        start_time = time.time()
        print(f"\nEpoch {epoch + 1}/{num_epochs}")

        # Training Phase
        model.train()
        batch_train_count = 0
        for x_batch, y_batch in train_dl:
            batch_train_count += 1
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)

            # Forward pass
            pred = model(x_batch)
            loss = loss_fn(pred, y_batch)

            # Backward pass
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            # Track metrics
            loss_hist_train[epoch] += loss.item() * y_batch.size(0)
            is_correct = (torch.argmax(pred, dim=1) == y_batch).float()
            accuracy_hist_train[epoch] += is_correct.sum().cpu()

            if batch_train_count % 10 == 0:  # Print progress every 10 batches
                print(f"  [Training] Batch {batch_train_count}/{len(train_dl)} - Loss: {loss.item():.4f}")

        # Calculate epoch-level metrics for training
        loss_hist_train[epoch] /= len(train_dl.dataset)
        accuracy_hist_train[epoch] /= len(train_dl.dataset)

        # Validation Phase
        model.eval()
        batch_valid_count = 0
        with torch.no_grad():
            for x_batch, y_batch in valid_dl:
                batch_valid_count += 1
                x_batch = x_batch.to(device)
                y_batch = y_batch.to(device)

                # Forward pass
                pred = model(x_batch)
                loss = loss_fn(pred, y_batch)

                # Track metrics
                loss_hist_valid[epoch] += loss.item() * y_batch.size(0)
                is_correct = (torch.argmax(pred, dim=1) == y_batch).float()
                accuracy_hist_valid[epoch] += is_correct.sum().cpu()

                if batch_valid_count % 10 == 0:  # Print progress every 10 batches
                    print(f"  [Validation] Batch {batch_valid_count}/{len(valid_dl)} - Loss: {loss.item():.4f}")

        # Calculate epoch-level metrics for validation
        loss_hist_valid[epoch] /= len(valid_dl.dataset)
        accuracy_hist_valid[epoch] /= len(valid_dl.dataset)

        end_time = time.time()
        epoch_duration = end_time - start_time

        # Early Stopping
        if epoch > 0 and loss_hist_valid[epoch] > min(loss_hist_valid[:epoch]):
            patience_counter += 1
        else:
            patience_counter = 0
            best_model_wts = model.state_dict()  # Save the best model weights

        if patience_counter >= patience:
            print(f"Early stopping at epoch {epoch + 1}")
            model.load_state_dict(best_model_wts)  # Load the best model weights
            break

        # Epoch Summary
        print(
            f"Epoch {epoch + 1}/{num_epochs} Summary:"
            f"\n  Training - Loss: {loss_hist_train[epoch]:.4f}, Accuracy: {accuracy_hist_train[epoch]:.4f}"
            f"\n  Validation - Loss: {loss_hist_valid[epoch]:.4f}, Accuracy: {accuracy_hist_valid[epoch]:.4f}"
            f"\n  Duration: {epoch_duration:.2f}s"
        )

    return loss_hist_train, loss_hist_valid, accuracy_hist_train, accuracy_hist_valid

# Training parameters
num_epochs = 300
hist = train(model, num_epochs, train_loader, val_loader, patience=5)


In [0]:
import matplotlib.pyplot as plt
import numpy as np
early_stop_epoch = next((i for i, v in enumerate(hist[0]) if v == 0), len(hist[0]))
x_arr = np.arange(len(hist[0])) + 1

# Adjust x_arr to stop at early_stop_epoch
x_arr = x_arr[:early_stop_epoch]

fig = plt.figure(figsize=(12, 4))
ax = fig.add_subplot(1, 2, 1)
ax.plot(x_arr, hist[0][:early_stop_epoch], '-o', label='Train loss')
ax.plot(x_arr, hist[1][:early_stop_epoch], '--<', label='Validation loss')
ax.set_xlabel('Epoch', size=15)
ax.set_ylabel('Loss', size=15)
ax.legend(fontsize=15)
ax = fig.add_subplot(1, 2, 2)
ax.plot(x_arr, hist[2][:early_stop_epoch], '-o', label='Train acc.')
ax.plot(x_arr, hist[3][:early_stop_epoch], '--<', label='Validation acc.')
ax.legend(fontsize=15)
ax.set_xlabel('Epoch', size=15)
ax.set_ylabel('Accuracy', size=15)

plt.show()

In [0]:
from sklearn.metrics import classification_report

# Zet het model in evaluatiemodus
model.eval()

# Lijsten om de echte labels en voorspellingen op te slaan
all_labels = []
all_preds = []

# Geen gradientberekeningen nodig tijdens evaluatie
with torch.no_grad():
    for x_batch, y_batch in test_loader:
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        
        # Voorspellingen maken
        preds = model(x_batch)
        preds = torch.argmax(preds, dim=1)
        
        # Voeg de echte labels en voorspellingen toe aan de lijsten
        all_labels.extend(y_batch.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())

# Genereer het classificatierapport
report = classification_report(all_labels, all_preds, target_names=['Crazing', 'Inclusion', 'Patches', 'Pitted', 'Rolled', 'Scratches'])
print(report)